In [1]:
import torch
dir_train = 'dataset/vlsp21/final_7900.pkl'
dir_dev = 'dataset/vlsp21/dev.pkl'
dir_test = 'dataset/vlsp21/test.pkl'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import time
from transformers import BertTokenizer, BertConfig, BertModel
from TorchCRF import CRF
import torch.nn as nn
import torch
import warnings
warnings.filterwarnings("ignore")
import pickle

from utils.bert_loader import BERT_DATALOADER
from utils.bert_evaluate import BERT_EVALUATE
from utils.bert_predict import BERT_PREDICT
from utils.bert_visualize import BERT_VISUALIZE
from utils.bert_model import BERT_4_CRF, BERT_4_SOFTMAX

In [3]:
start = time.time()
tokenizer = BertTokenizer.from_pretrained('mBert_Tokenizer', do_lower_case=False,use_fast=False)
config = BertConfig.from_pretrained('mBert_Config', output_hidden_states=True)
config.max_position_embeddings = 512

bert_model = BertModel.from_pretrained(
                        'mBert_Model',
                        config=config,
                        add_pooling_layer=False
)
print("Loading: ",time.time()-start, " S")

file mBert_Tokenizer\config.json not found


Loading:  5.003033876419067  S


## BERT_4_CRF

In [4]:
tag = ['ADDRESS','SKILL','EMAIL','PERSON','PHONENUMBER','MISCELLANEOUS','QUANTITY','PERSONTYPE',
              'ORGANIZATION','PRODUCT','IP','LOCATION','O','DATETIME','EVENT', 'URL']

In [5]:
def read_dataset(dir_train):
    with open(dir_train ,'rb') as f:
        _data = pickle.load(f)
    data = [sq for sq in _data if len(sq) >= 1]
    return data

### 1. Create Dataloader

In [6]:
data_train = read_dataset(dir_train)
TRAIN_SET = BERT_DATALOADER(data_train, tokenizer, tag, device)
train_dataloader = TRAIN_SET.create_dataloader()

data_dev = read_dataset(dir_dev)
DEV_SET = BERT_DATALOADER(data_dev, tokenizer, tag, device)
dev_dataloader = DEV_SET.create_dataloader()

data_test = read_dataset(dir_test)
TEST_SET = BERT_DATALOADER(data_test, tokenizer, tag, device)
test_dataloader = TEST_SET.create_dataloader()

### 2. Load model

In [7]:
PATH = 'model/16_bert_4_crf_ner_08602.pt'
#PATH = 'model/bert_4_crf.pt'
model = BERT_4_CRF(bert_model, num_labels=len(TRAIN_SET.tag2idx))
model.load_state_dict(torch.load(PATH), strict=False)
model.to(device)

BERT_4_CRF(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


### 3. Evaluate

In [8]:
BERT_EVALUATE(model=model, tokenizer=tokenizer, dataloader=dev_dataloader, tag_values= ['PAD']+tag, device = device, type_dataset='dev', model_type='crf')

Progress Bar: 100%|████████████████████████████████████████████████████████████████████| 81/81 [01:40<00:00,  1.24s/it]


Validation F1-Score: 0.6960845072266288
               precision    recall  f1-score   support

      ADDRESS     0.3167    0.5429    0.4000        35
       PERSON     0.8766    0.9519    0.9127      3075
  PHONENUMBER     0.4444    1.0000    0.6154         8
MISCELLANEOUS     0.4779    0.5085    0.4928       234
     QUANTITY     0.6029    0.8115    0.6918      2133
   PERSONTYPE     0.5432    0.8209    0.6538       843
 ORGANIZATION     0.7887    0.8237    0.8058      4214
      PRODUCT     0.4847    0.4830    0.4839      1149
     LOCATION     0.7769    0.8974    0.8328      3636
            O     0.9837    0.9540    0.9686    110861
     DATETIME     0.5875    0.8585    0.6976      1760
        EVENT     0.4768    0.6609    0.5540       699
          URL     0.8868    1.0000    0.9400        47

    micro avg     0.9370    0.9368    0.9369    128694
    macro avg     0.6344    0.7933    0.6961    128694
 weighted avg     0.9459    0.9368    0.9401    128694



In [9]:
BERT_EVALUATE(model=model, tokenizer=tokenizer, dataloader=test_dataloader, tag_values= ['PAD']+tag, device = device, type_dataset='test', model_type='crf')

Progress Bar: 100%|██████████████████████████████████████████████████████████████████| 133/133 [02:42<00:00,  1.22s/it]


Validation F1-Score: 0.681605220936032
               precision    recall  f1-score   support

      ADDRESS     0.2766    0.8125    0.4127        16
        EMAIL     1.0000    1.0000    1.0000         7
       PERSON     0.8969    0.9650    0.9297      6623
  PHONENUMBER     0.6000    0.5000    0.5455         6
MISCELLANEOUS     0.1645    0.2591    0.2013       247
     QUANTITY     0.8079    0.8045    0.8062      7185
   PERSONTYPE     0.6107    0.4004    0.4837      3651
 ORGANIZATION     0.7448    0.7360    0.7404      6571
      PRODUCT     0.4721    0.4286    0.4493      2886
     LOCATION     0.7429    0.8970    0.8127      3698
            O     0.9764    0.9714    0.9739    232378
     DATETIME     0.8517    0.9415    0.8943      4563
        EVENT     0.2317    0.6481    0.3413       449
          URL     0.9544    0.9486    0.9515       331

    micro avg     0.9450    0.9447    0.9448    268611
    macro avg     0.6665    0.7366    0.6816    268611
 weighted avg     0.9465

In [8]:
BERT_EVALUATE(model=model, tokenizer=tokenizer, dataloader=train_dataloader, tag_values= ['PAD']+tag, device = device, type_dataset='train', model_type='crf')

Progress Bar: 100%|██████████████████████████████████████████████████████████████████| 403/403 [08:18<00:00,  1.24s/it]


Validation F1-Score: 0.9623404645964808
               precision    recall  f1-score   support

      ADDRESS     0.9617    0.8559    0.9057      1145
     DATETIME     0.9773    0.9748    0.9760     15071
        EMAIL     1.0000    1.0000    1.0000       277
        EVENT     0.9544    0.9012    0.9270      7007
           IP     0.9978    1.0000    0.9989       461
     LOCATION     0.9693    0.9771    0.9732     21584
MISCELLANEOUS     0.9222    0.9118    0.9170      2380
            O     0.9939    0.9966    0.9952    551707
 ORGANIZATION     0.9733    0.9606    0.9669     23572
       PERSON     0.9902    0.9949    0.9925     16986
   PERSONTYPE     0.9411    0.9356    0.9383      8776
  PHONENUMBER     0.9953    1.0000    0.9977       637
      PRODUCT     0.9573    0.9039    0.9298      8456
     QUANTITY     0.9805    0.9653    0.9729     21402
        SKILL     0.9522    0.8743    0.9116       342
          URL     0.9902    0.9990    0.9946      1016

     accuracy          

### 3. Predicting

In [11]:
test_sentence = '''
Apple đứng đầu danh sách thương hiệu tốt nhất của Interbrand năm thứ 9 liên tiếp

Năm thứ 9 liên tiếp, công ty tư vấn thương hiệu toàn cầu Interbrand đã xếp Apple vào vị trí đầu danh sách các thương hiệu giá trị nhất thế giới.
Giá trị thương hiệu được công ty Interbrand định giá cho Apple là 408,6 tỷ USD. Con số này tăng 26% so với năm ngoái khi thương hiệu của Apple được định giá 323 tỷ USD.

Amazon và Microsoft chiếm vị trí thứ hai và thứ ba với mức định giá tương ứng là 249 tỷ USD và 210 tỷ USD trong khi Google và Samsung lọt vào top 5. Định giá chung của Apple, Amazon và Microsoft chiếm 62,3% tổng giá trị của 10 thương hiệu hàng đầu được đánh giá trong báo cáo năm nay.

"Apple đã đa dạng hóa sản phẩm hơn trong lĩnh vực chăm sóc sức khỏe với Apple Watch hiện ghi lại nồng độ oxy trong máu, dịch vụ đăng ký trong giải trí, lưu trữ dữ liệu và âm nhạc. Thương hiệu này tiếp tục gắn bó với khách hàng và tiếp tục phát triển mạnh mẽ" - Báo cáo của Interbrand viết - "Cuối cùng, Apple vẫn thể hiện hướng đi đáng chú ý và giá trị thương hiệu tập trung vào việc cung cấp cho người tiêu dùng trải nghiệm đơn giản, liền mạch. Điều này đã được khẳng định khi gia tăng 26% về giá trị thương hiệu".

Interbrand tính toán định giá thương hiệu như một thước đo sức mạnh thương hiệu, tính đến nhiều yếu tố bên trong và bên ngoài chủ quan như khả năng lãnh đạo, mức độ gắn kết và mức độ liên quan. Kết quả là đánh giá tác động của một công ty đối với khách hàng, nhân viên và nhà đầu tư. Interbrand cho biết, các thương hiệu mạnh tạo ra ảnh hưởng đối với người tiêu dùng, tạo ra một lượng khách hàng trung thành, thu hút và giữ chân nhân viên cũng như giảm chi phí tài chính.

Theo dữ liệu lịch sử của Interbrand, Apple bắt đầu giành được chỗ đứng trước các đối thủ cạnh tranh trong phân khúc công nghệ và các tên tuổi gia dụng như McDonalds và GE vào năm 2012. Apple đã vượt qua Coca-Cola, trở thành thương hiệu giá trị nhất thế giới vào năm 2013.

'''
prediction= BERT_PREDICT(model, tokenizer, ['PAD']+tag, device, test_sentence, 'crf')
BERT_VISUALIZE(prediction)

## Test model khác 

In [13]:
#PATH = 'model/16_bert_4_crf_ner_08602.pt'
PATH = 'model/16_bert_4_crf_ner.pt'
model = BERT_4_CRF(bert_model, num_labels=len(TRAIN_SET.tag2idx))
model.load_state_dict(torch.load(PATH), strict=False)
model.to(device)

BERT_4_CRF(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [14]:
BERT_EVALUATE(model=model, tokenizer=tokenizer, dataloader=dev_dataloader, tag_values= ['PAD']+tag, device = device, type_dataset='dev', model_type='crf')

Progress Bar: 100%|████████████████████████████████████████████████████████████████████| 81/81 [01:36<00:00,  1.20s/it]


Validation F1-Score: 0.7302254566170878
               precision    recall  f1-score   support

      ADDRESS     0.2705    0.9429    0.4204        35
       PERSON     0.9096    0.9395    0.9243      3075
  PHONENUMBER     1.0000    0.8750    0.9333         8
MISCELLANEOUS     0.5059    0.5513    0.5276       234
     QUANTITY     0.5898    0.8528    0.6973      2133
   PERSONTYPE     0.6062    0.7616    0.6751       843
 ORGANIZATION     0.7323    0.9141    0.8132      4214
      PRODUCT     0.5099    0.5135    0.5117      1149
     LOCATION     0.8301    0.8718    0.8504      3636
            O     0.9855    0.9520    0.9685    110861
     DATETIME     0.5811    0.8875    0.7023      1760
        EVENT     0.5017    0.6237    0.5561       699
          URL     0.8393    1.0000    0.9126        47

    micro avg     0.9383    0.9380    0.9382    128694
    macro avg     0.6817    0.8220    0.7302    128694
 weighted avg     0.9484    0.9380    0.9416    128694



In [15]:
BERT_EVALUATE(model=model, tokenizer=tokenizer, dataloader=test_dataloader, tag_values= ['PAD']+tag, device = device, type_dataset='test', model_type='crf')

Progress Bar: 100%|██████████████████████████████████████████████████████████████████| 133/133 [02:39<00:00,  1.20s/it]


Validation F1-Score: 0.678522406980939
               precision    recall  f1-score   support

      ADDRESS     0.1121    0.8125    0.1970        16
        EMAIL     1.0000    1.0000    1.0000         7
       PERSON     0.9431    0.9456    0.9444      6623
  PHONENUMBER     1.0000    0.5000    0.6667         6
MISCELLANEOUS     0.1801    0.3158    0.2294       247
     QUANTITY     0.7711    0.8335    0.8011      7185
   PERSONTYPE     0.6504    0.3613    0.4645      3651
 ORGANIZATION     0.6930    0.8471    0.7623      6571
      PRODUCT     0.4229    0.4161    0.4195      2886
     LOCATION     0.8414    0.8591    0.8501      3698
            O     0.9768    0.9670    0.9719    232378
     DATETIME     0.8211    0.9435    0.8780      4563
        EVENT     0.2537    0.6503    0.3650       449
          URL     0.9355    0.9637    0.9494       331

    micro avg     0.9435    0.9428    0.9431    268611
    macro avg     0.6858    0.7440    0.6785    268611
 weighted avg     0.9466

In [16]:
test_sentence = '''
Apple đứng đầu danh sách thương hiệu tốt nhất của Interbrand năm thứ 9 liên tiếp

Năm thứ 9 liên tiếp, công ty tư vấn thương hiệu toàn cầu Interbrand đã xếp Apple vào vị trí đầu danh sách các thương hiệu giá trị nhất thế giới.
Giá trị thương hiệu được công ty Interbrand định giá cho Apple là 408,6 tỷ USD. Con số này tăng 26% so với năm ngoái khi thương hiệu của Apple được định giá 323 tỷ USD.

Amazon và Microsoft chiếm vị trí thứ hai và thứ ba với mức định giá tương ứng là 249 tỷ USD và 210 tỷ USD trong khi Google và Samsung lọt vào top 5. Định giá chung của Apple, Amazon và Microsoft chiếm 62,3% tổng giá trị của 10 thương hiệu hàng đầu được đánh giá trong báo cáo năm nay.

"Apple đã đa dạng hóa sản phẩm hơn trong lĩnh vực chăm sóc sức khỏe với Apple Watch hiện ghi lại nồng độ oxy trong máu, dịch vụ đăng ký trong giải trí, lưu trữ dữ liệu và âm nhạc. Thương hiệu này tiếp tục gắn bó với khách hàng và tiếp tục phát triển mạnh mẽ" - Báo cáo của Interbrand viết - "Cuối cùng, Apple vẫn thể hiện hướng đi đáng chú ý và giá trị thương hiệu tập trung vào việc cung cấp cho người tiêu dùng trải nghiệm đơn giản, liền mạch. Điều này đã được khẳng định khi gia tăng 26% về giá trị thương hiệu".

Interbrand tính toán định giá thương hiệu như một thước đo sức mạnh thương hiệu, tính đến nhiều yếu tố bên trong và bên ngoài chủ quan như khả năng lãnh đạo, mức độ gắn kết và mức độ liên quan. Kết quả là đánh giá tác động của một công ty đối với khách hàng, nhân viên và nhà đầu tư. Interbrand cho biết, các thương hiệu mạnh tạo ra ảnh hưởng đối với người tiêu dùng, tạo ra một lượng khách hàng trung thành, thu hút và giữ chân nhân viên cũng như giảm chi phí tài chính.

Theo dữ liệu lịch sử của Interbrand, Apple bắt đầu giành được chỗ đứng trước các đối thủ cạnh tranh trong phân khúc công nghệ và các tên tuổi gia dụng như McDonalds và GE vào năm 2012. Apple đã vượt qua Coca-Cola, trở thành thương hiệu giá trị nhất thế giới vào năm 2013.

'''
prediction= BERT_PREDICT(model, tokenizer, ['PAD']+tag, device, test_sentence, 'crf')
BERT_VISUALIZE(prediction)

## BERT_4_SOFTMAX

In [ ]:
def read_dataset(dir_train):
    with open(dir_train ,'rb') as f:
        _data = pickle.load(f)
    return _data
tag = ['EMAIL', 'ADDRESS','PERSON','PHONENUMBER','MISCELLANEOUS','QUANTITY','PERSONTYPE',
              'ORGANIZATION','PRODUCT','SKILL','IP','LOCATION','O','DATETIME','EVENT','URL']

### 1. create Dataloader

In [ ]:
data_train = read_dataset(dir_train)
TRAIN_SET = BERT_DATALOADER(data_train, tokenizer, tag, device)
train_dataloader = TRAIN_SET.create_dataloader()

data_dev = read_dataset(dir_dev)
DEV_SET = BERT_DATALOADER(data_dev, tokenizer, tag, device)
dev_dataloader = DEV_SET.create_dataloader()

data_test = read_dataset(dir_test)
TEST_SET = BERT_DATALOADER(data_test, tokenizer, tag, device)
test_dataloader = TEST_SET.create_dataloader()

### 2. Load model

In [ ]:
PATH = 'model/bert_4_softmax.pt'
model = BERT_4_SOFTMAX(bert_model, num_labels=len(TRAIN_SET.tag2idx))
model.load_state_dict(torch.load(PATH), strict=False)
model.to(device)

### 3. Evaluate

In [ ]:
BERT_EVALUATE(model=model, tokenizer=tokenizer, dataloader=dev_dataloader, tag_values= tag+['PAD'], device = device, type_dataset='dev', model_type='softmax')

In [ ]:
BERT_EVALUATE(model=model, tokenizer=tokenizer, dataloader=test_dataloader, tag_values= tag+['PAD'], device = device, type_dataset='test', model_type='softmax')

### 3. Predicting

In [ ]:
test_sentence = '''
Apple đứng đầu danh sách thương hiệu tốt nhất của Interbrand năm thứ 9 liên tiếp

Năm thứ 9 liên tiếp, công ty tư vấn thương hiệu toàn cầu Interbrand đã xếp Apple vào vị trí đầu danh sách các thương hiệu giá trị nhất thế giới.
Giá trị thương hiệu được công ty Interbrand định giá cho Apple là 408,6 tỷ USD. Con số này tăng 26% so với năm ngoái khi thương hiệu của Apple được định giá 323 tỷ USD.

Amazon và Microsoft chiếm vị trí thứ hai và thứ ba với mức định giá tương ứng là 249 tỷ USD và 210 tỷ USD trong khi Google và Samsung lọt vào top 5. Định giá chung của Apple, Amazon và Microsoft chiếm 62,3% tổng giá trị của 10 thương hiệu hàng đầu được đánh giá trong báo cáo năm nay.

"Apple đã đa dạng hóa sản phẩm hơn trong lĩnh vực chăm sóc sức khỏe với Apple Watch hiện ghi lại nồng độ oxy trong máu, dịch vụ đăng ký trong giải trí, lưu trữ dữ liệu và âm nhạc. Thương hiệu này tiếp tục gắn bó với khách hàng và tiếp tục phát triển mạnh mẽ" - Báo cáo của Interbrand viết - "Cuối cùng, Apple vẫn thể hiện hướng đi đáng chú ý và giá trị thương hiệu tập trung vào việc cung cấp cho người tiêu dùng trải nghiệm đơn giản, liền mạch. Điều này đã được khẳng định khi gia tăng 26% về giá trị thương hiệu".

Interbrand tính toán định giá thương hiệu như một thước đo sức mạnh thương hiệu, tính đến nhiều yếu tố bên trong và bên ngoài chủ quan như khả năng lãnh đạo, mức độ gắn kết và mức độ liên quan. Kết quả là đánh giá tác động của một công ty đối với khách hàng, nhân viên và nhà đầu tư. Interbrand cho biết, các thương hiệu mạnh tạo ra ảnh hưởng đối với người tiêu dùng, tạo ra một lượng khách hàng trung thành, thu hút và giữ chân nhân viên cũng như giảm chi phí tài chính.

Theo dữ liệu lịch sử của Interbrand, Apple bắt đầu giành được chỗ đứng trước các đối thủ cạnh tranh trong phân khúc công nghệ và các tên tuổi gia dụng như McDonalds và GE vào năm 2012. Apple đã vượt qua Coca-Cola, trở thành thương hiệu giá trị nhất thế giới vào năm 2013.

'''
prediction= BERT_PREDICT(model, tokenizer, tag, device, test_sentence, 'softmax')
BERT_VISUALIZE(prediction)